# Timelapse 1-count analysis
## Used for estimating gain_model

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
# Uncomment this to easily reset
# from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
import os
os.environ["MPLCONFIGDIR"] = "/tmp"

In [ ]:
# @IMPORT-MERGE
import numpy as np
import pandas as pd
import itertools
import cv2
import random
from scipy.stats import norm, lognorm
from IPython.display import HTML, display
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.run.sigproc_v2.sigproc_v2_result import df_filter, radmat_from_df_filter, df_to_radmat
from plaster.run.plots import plots, plots_dev
from plaster.run.plots.plots_sigproc import plot_psfs, circle_locs, sigproc_v2_im, sigproc_v2_movie_from_df, sigproc_v2_im_from_df
from plaster.run.plots.plots_sigproc import wizard_xy_df, wizard_scat_df, wizard_raw_images
from plaster.run.sigproc_v2.synth import Synth
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.utils import data
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.utils import data
from plaster.tools.calibration.calibration import Calibration
from plaster.tools.ipynb_helpers.displays import hd, movie
z = zplots.setup()

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/erisyon/internal/jobs_folder/abbe7_1t")
run = job.runs[0]

In [ ]:
n_channels = run.sigproc_v2.n_channels
n_cycles = run.sigproc_v2.n_cycles
sigproc_df = run.sigproc_v2.fields__n_peaks__peaks__radmat()
assert n_channels == 1
sig = radmat_from_df_filter(sigproc_df, channel_i=0)

# TEMPORARY HACK
sig = sig[:, 0:25]    
    
z.im_clus(sig, _cspan=(-1000, 20000))

# Estimate zero gain model

In [ ]:
def hist_peak(sig, bins):
    """
    Fit a curve to a historgram to get an estimate of the center of the dominant peak
    using a savgol_filter.
    See https://stackoverflow.com/a/20642478
    """
    from scipy.signal import savgol_filter
    _hist, _edges = np.histogram(sig, bins=bins)
    filt = savgol_filter((_edges[1:], _hist), 101, 3)
    x = filt[0]
    y = filt[1]
    return x[np.argmax(y)], np.max(_hist), y

In [ ]:
# Estimate zero gain by examining the histogram near zero
filt_sig = sig[sig != 0]
bins = np.linspace(-2000, 4000, 1000)
zero_beta, top, smooth = hist_peak(filt_sig, bins)

# Assume that the samples to the left of this peak are un-contaminated
# by signal -- so we can take these, flip the around, and compute
# a standard deviation to get zero_sigma
left = filt_sig[filt_sig < zero_beta]
left = np.concatenate((left, 2*zero_beta-left))
zero_sigma = np.std(left)

# PLOT the results
with z(_merge=True):
    z.hist(filt_sig, _bins=bins, f_x_axis_label="inten", f_title=f"field uncorrected", _size=500)
    z.line(x=bins[1:], y=smooth, color="orange")
    z.line(x=[zero_beta, zero_beta], y=[0, top], color="red")
    z.line(x=[zero_beta-zero_sigma, zero_beta-zero_sigma], y=[0, top], color="green")
    z.line(x=[zero_beta+zero_sigma, zero_beta+zero_sigma], y=[0, top], color="green")

# Estimate beta and sigma

In [ ]:
dark = zero_beta + 7 * zero_sigma
debug(dark)

# Balance every row by masking out the dark elements
# and removing "one-hit wonders" and "remainders" as those
# are both likely to be contamination
filt_sig = sig.copy()
filt_sig = filt_sig[(filt_sig[:, 1] > dark) & (filt_sig[:, -1] < dark)]
filt_sig[filt_sig < dark] = np.nan
filt_sig = filt_sig[np.any(~np.isnan(filt_sig), axis=1)]

row_means = np.nanmean(filt_sig, axis=1)
row_means = row_means / np.mean(row_means)
balanced = filt_sig / row_means[:, None]

stack_im = np.hstack((filt_sig, balanced))

balanced = balanced[~np.isnan(balanced)]
log_bal = np.log(balanced)
bins = np.linspace(7, 11, 1000)
beta, sigma = norm.fit(log_bal)
beta = np.exp(beta)

with z(_cols=2):
    z.im_clus(np.nan_to_num(stack_im))
    model_samples = lognorm.rvs(scale=beta, s=sigma, size=len(balanced))
    with z(_merge=True, _bins=(0, 20_000, 1000), alpha=0.3):
        z.hist(balanced, color="blue")
        z.hist(model_samples, color="red")

print(f"""
    dark      = {dark:>9.3f}
    zero_beta = {zero_beta:>9.3f}
    zero_sigma= {zero_sigma:>9.3f}
    beta      = {beta:>9.3f}
    sigma     = {sigma:>9.3f}
""")